In [133]:
import pandas as pd
import numpy as np

In [135]:
train_df = pd.read_csv('../data/train_data.csv')
val_df = pd.read_csv('../data/validation_data.csv')
test_df = pd.read_csv('../data/test_data.csv')

In [136]:
train_df.head()

,CategoryCode,ItemCode,DateID,DailySales
0,category_2,117610,11/6/2021,7
1,category_4,836584,11/18/2021,16
2,category_1,370195,1/24/2022,6
3,category_2,172582,10/30/2021,5
4,category_2,1006009,10/30/2021,5


In [137]:
train_df.shape

(19921, 4)

In [138]:
train_udf = train_df.copy().iloc[:, 1:4].sort_values(by=['ItemCode', 'DateID'])
train_udf.head()

,ItemCode,DateID,DailySales
10815,3418,1/1/2022,4
13878,3418,1/10/2022,7
6625,3418,1/11/2022,7
12374,3418,1/12/2022,14
6378,3418,1/13/2022,28


In [139]:
start_date = min(train_udf.DateID)
end_date = max(train_udf.DateID)
# print(start_date, end_date)
def df_func(x):
    _x = x.copy().reset_index(drop=True)
    _x.drop(columns=['ItemCode'], inplace=True)
    _x['DateID'] = pd.to_datetime(_x['DateID'])
    _x = _x.set_index('DateID')
    __x = _x.resample('D').sum().fillna(0)
    # print(__x)
    return __x

In [141]:
res_df = train_udf.groupby('ItemCode').apply(lambda x: df_func(x)).reset_index().sort_values(by=['ItemCode', 'DateID'])
# train_udf.groupby('ItemCode').apply(lambda x: print(x.index)).reset_index()
res_df.head()

,ItemCode,DateID,DailySales
0,3418,2021-10-01,18
1,3418,2021-10-02,3
2,3418,2021-10-03,8
3,3418,2021-10-04,2
4,3418,2021-10-05,4


In [142]:
I2C_map = pd.Series(train_df.CategoryCode.values, index=train_df.ItemCode).to_dict()
res_df['CategoryCode'] = res_df['ItemCode'].map(I2C_map)

In [146]:
res_df.columns = ['ItemCode', 'Date', 'DailySales', 'CategoryCode']
res_df = res_df.sort_values(by=['ItemCode', 'Date'])
res_df.head()

,ItemCode,Date,DailySales,CategoryCode
0,3418,2021-10-01,18,category_1
1,3418,2021-10-02,3,category_1
2,3418,2021-10-03,8,category_1
3,3418,2021-10-04,2,category_1
4,3418,2021-10-05,4,category_1


In [149]:
res_df.to_csv('../data/train_dataV6.csv')

---
Make list

In [147]:
_df1 = res_df.copy().loc[:, ['ItemCode', 'DailySales']].groupby('ItemCode').agg(list).reset_index()
_df1.head()

,ItemCode,DailySales
0,3418,"[18, 3, 8, 2, 4, 2, 8, 12, 0, 14, 2, 9, 8, 5, ..."
1,3427,"[2, 7, 2, 3, 9, 0, 5, 8, 10, 5, 5, 4, 0, 4, 3,..."
2,7666,"[14, 58, 12, 8, 5, 33, 11, 23, 24, 15, 41, 20,..."
3,9925,"[1, 3, 1, 2, 0, 0, 2, 1, 0, 1, 8, 0, 2, 0, 0, ..."
4,16936,"[4, 1, 4, 1, 0, 0, 21, 2, 1, 0, 5, 9, 2, 8, 8,..."


In [148]:
_df1['CategoryCode'] = res_df['ItemCode'].map(I2C_map)
_df1.head()

,ItemCode,DailySales,CategoryCode
0,3418,"[18, 3, 8, 2, 4, 2, 8, 12, 0, 14, 2, 9, 8, 5, ...",category_1
1,3427,"[2, 7, 2, 3, 9, 0, 5, 8, 10, 5, 5, 4, 0, 4, 3,...",category_1
2,7666,"[14, 58, 12, 8, 5, 33, 11, 23, 24, 15, 41, 20,...",category_1
3,9925,"[1, 3, 1, 2, 0, 0, 2, 1, 0, 1, 8, 0, 2, 0, 0, ...",category_1
4,16936,"[4, 1, 4, 1, 0, 0, 21, 2, 1, 0, 5, 9, 2, 8, 8,...",category_1


In [150]:
_df1.to_csv('../data/DSalesPointsV2.csv', index=False)

***

In [151]:
train_items = set(train_df['ItemCode'])
val_items = set(val_df['ItemCode'])
test_items = set(test_df['ItemCode'])

In [152]:
stage_map = {i: 'validation' for i in val_items}
stage_map.update({i: 'test' for i in test_items})
stage_map.update({836584: 'train-only', 1090249: 'train-only'})

In [154]:
df2 = res_df.copy()
df2.head()

,ItemCode,Date,DailySales,CategoryCode
0,3418,2021-10-01,18,category_1
1,3418,2021-10-02,3,category_1
2,3418,2021-10-03,8,category_1
3,3418,2021-10-04,2,category_1
4,3418,2021-10-05,4,category_1


In [155]:
sales_point = df2.groupby('ItemCode')['DailySales'].count().reset_index()
sales = df2.groupby('ItemCode')['DailySales'].agg(list)
sales_point['stage'] = sales_point['ItemCode'].map(stage_map)
sales_point['sales'] = sales_point['ItemCode'].map(sales)
sales_point.columns = ['ItemCode', '#sales', 'stage', 'DailySales']
sales_point.head()
# sales_point1.to_csv('../data/SalesPointsV1.csv')

,ItemCode,#sales,stage,DailySales
0,3418,136,validation,"[18, 3, 8, 2, 4, 2, 8, 12, 0, 14, 2, 9, 8, 5, ..."
1,3427,136,validation,"[2, 7, 2, 3, 9, 0, 5, 8, 10, 5, 5, 4, 0, 4, 3,..."
2,7666,135,validation,"[14, 58, 12, 8, 5, 33, 11, 23, 24, 15, 41, 20,..."
3,9925,133,test,"[1, 3, 1, 2, 0, 0, 2, 1, 0, 1, 8, 0, 2, 0, 0, ..."
4,16936,136,test,"[4, 1, 4, 1, 0, 0, 21, 2, 1, 0, 5, 9, 2, 8, 8,..."


In [ ]:
sales_point.to_csv('../data/DSalesPointsV2.csv', index=False)

***

In [156]:
df2['WoY'] = df2["Date"].dt.weekofyear
df2.head()

C:\Users\deela\AppData\Local\Temp/ipykernel_41400/2620650257.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df2['WoY'] = df2["Date"].dt.weekofyear


,ItemCode,Date,DailySales,CategoryCode,WoY
0,3418,2021-10-01,18,category_1,39
1,3418,2021-10-02,3,category_1,39
2,3418,2021-10-03,8,category_1,39
3,3418,2021-10-04,2,category_1,40
4,3418,2021-10-05,4,category_1,40


In [157]:
df3 = df2.loc[:, ['ItemCode', 'DailySales', 'WoY']]
df3.head()

,ItemCode,DailySales,WoY
0,3418,18,39
1,3418,3,39
2,3418,8,39
3,3418,2,40
4,3418,4,40


In [76]:
def df_func1(x):
    _x = x.copy().reset_index(drop=True)
    _x.drop(columns=['ItemCode'], inplace=True)
    print(_x)
    __x = _x.groupby('WoYY').sum().reset_index()
    print(__x)
    return __x

In [79]:
pd.set_option('display.max_rows', None)
df4 = df3.groupby('ItemCode').apply(lambda x: df_func1(x))
df4.head()

WoY  DailySales
ItemCode                   
3418     0    1          83
         1    2          85
         2    3          89
         3    4          67
         4    5         109

In [158]:
df5 = df3.groupby(['ItemCode', 'WoY']).sum().reset_index()
df5.head()
# df5.to_csv('../data/train_dataV5.csv')

,ItemCode,WoY,DailySales
0,3418,1,83
1,3418,2,85
2,3418,3,89
3,3418,4,67
4,3418,5,109


In [160]:
I2C_map = pd.Series(train_df.CategoryCode.values,index=train_df.ItemCode).to_dict()
df5['CategoryCode'] = df5['ItemCode'].map(I2C_map)
df5.columns = ['ItemCode', 'WoY', 'WeeklySales', 'CategoryCode']
df5.head()

,ItemCode,WoY,WeeklySales,CategoryCode
0,3418,1,83,category_1
1,3418,2,85,category_1
2,3418,3,89,category_1
3,3418,4,67,category_1
4,3418,5,109,category_1


In [161]:
df6 = df5.copy().loc[:, ['CategoryCode', 'ItemCode', 'WoY', 'WeeklySales']]
df6.head()

,CategoryCode,ItemCode,WoY,WeeklySales
0,category_1,3418,1,83
1,category_1,3418,2,85
2,category_1,3418,3,89
3,category_1,3418,4,67
4,category_1,3418,5,109


In [102]:
df6.to_csv('../data/sample_train1.csv')

In [81]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3831 entries, 0 to 3830
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   ItemCode    3831 non-null   int64
 1   WoY         3831 non-null   int64
 2   DailySales  3831 non-null   int64
dtypes: int64(3)
memory usage: 89.9 KB


In [162]:
df5['WoY'].unique()

array([ 1,  2,  3,  4,  5,  6, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
       50, 51, 52], dtype=int64)

In [163]:
woy_map = {n: i for i,n in enumerate(range(39, 53), 1)}
woy_map.update({n: i for i,n in enumerate(range(1, 7), 15)})
woy_map

{39: 1,
 40: 2,
 41: 3,
 42: 4,
 43: 5,
 44: 6,
 45: 7,
 46: 8,
 47: 9,
 48: 10,
 49: 11,
 50: 12,
 51: 13,
 52: 14,
 1: 15,
 2: 16,
 3: 17,
 4: 18,
 5: 19,
 6: 20}

In [164]:
df5['WoY'] = df5['WoY'].map(woy_map)
df5.head()

,ItemCode,WoY,WeeklySales,CategoryCode
0,3418,15,83,category_1
1,3418,16,85,category_1
2,3418,17,89,category_1
3,3418,18,67,category_1
4,3418,19,109,category_1


In [168]:
sales_point1 = df5.groupby('ItemCode')['WeeklySales'].count().reset_index()
sales = df5.groupby('ItemCode')['WeeklySales'].agg(list)
sales_point1['stage'] = sales_point1['ItemCode'].map(stage_map)
sales_point1['sales'] = sales_point1['ItemCode'].map(sales)
sales_point1.columns = ['ItemCode', '#sales', 'stage', 'WeeklySales']
sales_point1.head()
sales_point1.to_csv('../data/WSalesPointsV2.csv')

In [33]:
df4 = df3.groupby('ItemCode').apply(lambda x: df_func1(x))
df4.head()

    DailySales     WoYY
0           14  2021-39
1           58  2021-39
2           12  2021-39
3            8  2021-40
4            5  2021-40
..         ...      ...
35           0  2021-44
36           0  2021-44
37           4  2021-44
38           5  2021-45
39           3  2021-45

[40 rows x 2 columns]
       WoYY  DailySales
0   2021-39          84
1   2021-40         119
2   2021-41         196
3   2021-42         185
4   2021-43          94
5   2021-44          15
6   2021-45          10
7   2021-46          30
8   2021-47          79
9   2021-48          82
10  2021-49         141
11  2021-50         219
12  2021-51         125
13  2021-52          82
14   2022-1          15
15   2022-2          26
16   2022-3          28
17   2022-4          38
18   2022-5          41
19  2022-52          25
20   2022-6          39


WoYY  DailySales
ItemCode                       
3418     0  2021-39          29
         1  2021-40          42
         2  2021-41          41
         3  2021-42          41
         4  2021-43          44

In [100]:
df4.to_csv('../data/train_dataV4.csv')

In [98]:
val_df = pd.read_csv('../data/validation_data.csv')
test_df = pd.read_csv('../data/test_data.csv')

In [103]:
val_df.head()

,CategoryCode,ItemCode,Week,WeeklySales
0,category_2,1044502,w1,11
1,category_2,1105009,w1,11
2,category_2,913561,w4,5
3,category_1,1048975,w4,30
4,category_1,17287,w2,60


In [105]:
week_map = {
    'w1': 21,
    'w2': 22,
    'w3': 23,
    'w4': 24
}

In [106]:
val_df['Week'] = val_df['Week'].map(week_map)
test_df['Week'] = test_df['Week'].map(week_map)

val_df.head()

,CategoryCode,ItemCode,Week,WeeklySales
0,category_2,1044502,21,11
1,category_2,1105009,21,11
2,category_2,913561,24,5
3,category_1,1048975,24,30
4,category_1,17287,22,60


In [107]:
test_df.head()

,CategoryCode,ItemCode,Week,PredictedSales
0,category_1,43738,24,NaN
1,category_2,1006090,21,NaN
2,category_2,1076929,24,NaN
3,category_1,1081321,23,NaN
4,category_2,216151,24,NaN


In [108]:
val_df.columns = ['CategoryCode', 'ItemCode', 'WoY', 'WeeklySales']
test_df.columns = ['CategoryCode', 'ItemCode', 'WoY', 'WeeklySales']

In [109]:
val_df.to_csv('../data/sample_val1.csv')
test_df.to_csv('../data/sample_test1.csv')

ModuleNotFoundError: No module named 'kats'